<a href="https://colab.research.google.com/github/MariannaMois/MariannaMois/blob/main/dekor_sell_out_class_ipynb%22%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание проекта
Предоставлены исторические данные парсинга (MPStats) о продажах на маркетплейсах. Требуется классифицировать данные согласно внутреннему классификатору продукции компании. Разделы классификатора имеют множественные пересечения, ошибки и задвоения классов, так как выделены на основе описания товаров компании маркетологами.
Исходные данные парсинга содержат смежные и ошибочные группы товаров, неточности, описания несоответствующие классам.

Цель проекта — обработка и категоризация данных, а также удаление нежелательных категорий товаров.

Задачи проекта:

1.   Загрузка и предобработка данных.
2.   Использование предобученной модели для классификации товаров.
3.   Применение правил для фильтрации и удаления нежелательных категорий.
4.   Сохранение результатов в файл Excel для дальнейшего анализа.


# Подключение и установка библиотек


На данном этапе подключается Google Drive для работы с файлами, а также устанавливаются библиотеки для обработки текста и работы с Excel.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files

In [ ]:
!pip install spacy
!pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import spacy

In [ ]:
!python -m spacy download ru_core_news_sm
nlp = spacy.load('ru_core_news_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 41.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Загрузка модели и данных

Данные загружаются из CSV-файла, расположенного в Google Drive. Для удобства анализа выводятся первые строки таблицы.

In [ ]:
model_save_path = '/content/drive/My Drive/Colab Notebooks/RUS_dekor_NEW_best_model.keras'
loaded_model = tf.keras.models.load_model(model_save_path)

In [ ]:
file_path = '/content/drive/MyDrive/time/Ноябрь.csv'

test = pd.read_csv(file_path, delimiter=';')

<ipython-input-15-e3c2d1f23557>:3: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(file_path, delimiter=';')


In [ ]:
test

,Фото,Название,SKU,Предмет,Продавец,Бренд,Ссылка,ID склейки,Цвет,Страна,...,Цена со скидкой,Цена с WB кошельком,Ср. цена,Медиан. цена,Дробный рейтинг,Отзывы,Дней на сайте,Кол-во акционных категорий,Дата обнаружения,Дата первого отзыва
0,https://basket-13.wbbasket.ru/vol1972/part1972...,Удлиняющая термотушь с эффектом роста ресниц,197297783,Красота / Туши,ООО Алмеа,Xlash,https://www.wildberries.ru/catalog/197297783/d...,29484580,"глубокий черный, черный матовый, черный",Великобритания,...,24840,24094,"25204,54",25205,"4,6",1644,295,2,2024-02-09,2022-02-10
1,https://basket-04.wbbasket.ru/vol515/part51524...,Гель для бровей фиксирующий стойкий прозрачный...,51524888,Красота / Гели для бровей,ООО Дрожь Бьюти,PUSY,https://www.wildberries.ru/catalog/51524888/de...,188507080,NaN,Россия,...,532,532,"551,42",551,"4,7",175064,1083,3,2021-12-13,2022-10-21
2,https://basket-07.wbbasket.ru/vol1105/part1105...,"Тушь для ресниц черная объем и удлинение, 11 г",110572701,Красота / Туши,ООО Космо Бьюти,Lavant,https://www.wildberries.ru/catalog/110572701/d...,100723030,черный,Беларусь,...,394,394,"372,53",416,"4,7",139081,890,3,2022-06-24,2022-10-27
3,https://basket-10.wbbasket.ru/vol1420/part1420...,Энзимная пудра-пилинг для лица и тела с витами...,142044104,Красота / Пудры,ООО Рокет Лаунч,VOIS,https://www.wildberries.ru/catalog/142044104/d...,121070050,розовый,Китай,...,709,709,"727,84",762,"4,8",14923,538,0,2023-06-11,2023-06-19
4,https://basket-15.wbbasket.ru/vol2373/part2373...,Гель для бровей водостойкий BROW LAMINATOR EXT...,237336389,Красота / Гели для бровей,Диталир,LUXVISAGE,https://www.wildberries.ru/catalog/237336389/d...,14093014,прозрачный,Беларусь,...,291,291,291,291,"4,7",228571,169,3,2024-06-14,2021-11-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447349,https://basket-16.wbbasket.ru/vol2488/part2488...,Кушон для лица,248853558,Красота / Тональные кремы,ООО Аруу Групп,Косметика,https://www.wildberries.ru/catalog/248853558/d...,224433209,натуральный,Китай,...,213,213,0,225,"4,4",18,118,0,2024-08-04,2024-08-29
447350,https://basket-10.wbbasket.ru/vol1590/part1590...,Щеточки для бровей и ресниц,159002298,Красота / Щетки косметические,ИП Цыганкова А О,Synthetic Creator,https://www.wildberries.ru/catalog/159002298/d...,166659403,красный,Китай,...,150,150,0,150,"4,9",518,574,0,2023-05-06,2023-06-07
447351,https://basket-15.wbbasket.ru/vol2241/part2241...,Увлажняющая легкая пудра для лица Корея матиру...,224116518,Красота / Пудры,ИП Миллионерши,q,https://www.wildberries.ru/catalog/224116518/d...,201773169,медово-бежевый,Республика Корея,...,441,427,0,945,5,1,230,0,2024-04-14,2024-04-22
447352,https://basket-17.wbbasket.ru/vol2791/part2791...,Фиксатор для макияжа матирующий Matte 100 мл,279138618,Красота / Фиксаторы макияжа,ООО Добрый Вечер,Influence Beauty,https://www.wildberries.ru/catalog/279138618/d...,259004596,NaN,NaN,...,621,621,0,560,0,0,6,0,2024-11-24,1970-01-01


# Предобработка данных
Для предобработки текста используется библиотека SpaCy. Текст очищается от стоп-слов, приводится к нижнему регистру и лемматизируется.

In [ ]:
test['brand_old'] = test['Бренд']
test['text'] = test['Название']

In [ ]:
test['brand_old'] = test['brand_old'].apply(lambda x: 'Нет бренда' if isinstance(x, str) and 'нет бренда' in x.lower() else x)

In [ ]:
test

,Фото,Название,SKU,Предмет,Продавец,Бренд,Ссылка,ID склейки,Цвет,Страна,...,Ср. цена,Медиан. цена,Дробный рейтинг,Отзывы,Дней на сайте,Кол-во акционных категорий,Дата обнаружения,Дата первого отзыва,brand_old,text
0,https://basket-13.wbbasket.ru/vol1972/part1972...,Удлиняющая термотушь с эффектом роста ресниц,197297783,Красота / Туши,ООО Алмеа,Xlash,https://www.wildberries.ru/catalog/197297783/d...,29484580,"глубокий черный, черный матовый, черный",Великобритания,...,"25204,54",25205,"4,6",1644,295,2,2024-02-09,2022-02-10,Xlash,Удлиняющая термотушь с эффектом роста ресниц
1,https://basket-04.wbbasket.ru/vol515/part51524...,Гель для бровей фиксирующий стойкий прозрачный...,51524888,Красота / Гели для бровей,ООО Дрожь Бьюти,PUSY,https://www.wildberries.ru/catalog/51524888/de...,188507080,NaN,Россия,...,"551,42",551,"4,7",175064,1083,3,2021-12-13,2022-10-21,PUSY,Гель для бровей фиксирующий стойкий прозрачный...
2,https://basket-07.wbbasket.ru/vol1105/part1105...,"Тушь для ресниц черная объем и удлинение, 11 г",110572701,Красота / Туши,ООО Космо Бьюти,Lavant,https://www.wildberries.ru/catalog/110572701/d...,100723030,черный,Беларусь,...,"372,53",416,"4,7",139081,890,3,2022-06-24,2022-10-27,Lavant,"Тушь для ресниц черная объем и удлинение, 11 г"
3,https://basket-10.wbbasket.ru/vol1420/part1420...,Энзимная пудра-пилинг для лица и тела с витами...,142044104,Красота / Пудры,ООО Рокет Лаунч,VOIS,https://www.wildberries.ru/catalog/142044104/d...,121070050,розовый,Китай,...,"727,84",762,"4,8",14923,538,0,2023-06-11,2023-06-19,VOIS,Энзимная пудра-пилинг для лица и тела с витами...
4,https://basket-15.wbbasket.ru/vol2373/part2373...,Гель для бровей водостойкий BROW LAMINATOR EXT...,237336389,Красота / Гели для бровей,Диталир,LUXVISAGE,https://www.wildberries.ru/catalog/237336389/d...,14093014,прозрачный,Беларусь,...,291,291,"4,7",228571,169,3,2024-06-14,2021-11-20,LUXVISAGE,Гель для бровей водостойкий BROW LAMINATOR EXT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447349,https://basket-16.wbbasket.ru/vol2488/part2488...,Кушон для лица,248853558,Красота / Тональные кремы,ООО Аруу Групп,Косметика,https://www.wildberries.ru/catalog/248853558/d...,224433209,натуральный,Китай,...,0,225,"4,4",18,118,0,2024-08-04,2024-08-29,Косметика,Кушон для лица
447350,https://basket-10.wbbasket.ru/vol1590/part1590...,Щеточки для бровей и ресниц,159002298,Красота / Щетки косметические,ИП Цыганкова А О,Synthetic Creator,https://www.wildberries.ru/catalog/159002298/d...,166659403,красный,Китай,...,0,150,"4,9",518,574,0,2023-05-06,2023-06-07,Synthetic Creator,Щеточки для бровей и ресниц
447351,https://basket-15.wbbasket.ru/vol2241/part2241...,Увлажняющая легкая пудра для лица Корея матиру...,224116518,Красота / Пудры,ИП Миллионерши,q,https://www.wildberries.ru/catalog/224116518/d...,201773169,медово-бежевый,Республика Корея,...,0,945,5,1,230,0,2024-04-14,2024-04-22,q,Увлажняющая легкая пудра для лица Корея матиру...
447352,https://basket-17.wbbasket.ru/vol2791/part2791...,Фиксатор для макияжа матирующий Matte 100 мл,279138618,Красота / Фиксаторы макияжа,ООО Добрый Вечер,Influence Beauty,https://www.wildberries.ru/catalog/279138618/d...,259004596,NaN,NaN,...,0,560,0,0,6,0,2024-11-24,1970-01-01,Influence Beauty,Фиксатор для макияжа матирующий Matte 100 мл


In [ ]:
test['brand_old'].fillna('Нет бренда', inplace=True)
test['brand_old'].replace(0, 'Нет бренда', inplace=True)

<ipython-input-21-5e4cdec4fa99>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['brand_old'].fillna('Нет бренда', inplace=True)


In [ ]:
def preprocess_text_spacy(text):
    doc = nlp(str(text).lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

In [ ]:
test['Processed_text'] = test['text'].apply(preprocess_text_spacy)

# Классификация товаров с использованием предобученной модели
На данном этапе используется предобученная модель для классификации товаров. Текстовые данные преобразуются в числовые последовательности с помощью токенизатора, после чего модель предсказывает кластер для каждого товара.

In [ ]:
num_words = 20000
max_news_len = 30
nb_classes = 64

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/RUS_dekor_NEW_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test['Processed_text'])
x_test = pad_sequences(test_sequences, maxlen=max_news_len)

In [ ]:
cluster_predictions = loaded_model.predict(x_test)
cluster_predictions = np.argmax(cluster_predictions, axis=1)
test['cluster'] = cluster_predictions

13980/13980 ━━━━━━━━━━━━━━━━━━━━ 209s 15ms/step


In [ ]:
learn_df_new = pd.read_excel('/content/drive/MyDrive/Dekor_NEW/Learn.xlsx', sheet_name='Group', usecols=['cluster', 'ГРУППА', 'ПОДГРУППА', 'КАТЕГОРИЯ', 'ПОДКАТЕГОРИЯ'])

In [ ]:
df_updated_with_learn = test.merge(learn_df_new, how='left', on='cluster')

# Фильтрация данных и удаление нежелательных категорий
Данные фильтруются: нежелательные категории товаров удаляются на основе заранее заданного списка.

In [ ]:
df_updated_with_learn['text'] = df_updated_with_learn['text'].astype(str)

In [ ]:
keywords = [
    "размет", "для удаления", "линейка", "эскиз", "депиляц", "перманентного тату", "нейтрализатор Thuya", "коврик для бровиста",
    "обезжириват", "раствор", "для ламинирования", "для завивки",
    "валики", "для наращивания", "эскиз", "ремув", "для перманент", "для татуажа", "оксид", "нейтрализатор", "окислитель",
]

In [ ]:
def update_row(row):
    # Приведение к нижнему регистру для проверки
    товар_lower = row['text'].lower()

    # Проверка наличия любого из ключевых слов с дополнительными условиями
    if (
        ("очищающ" in товар_lower and "тональный" not in товар_lower) or
        ("состав" in товар_lower and "натуральный" not in товар_lower) or
        ("для ланинирования" in товар_lower and "гель" not in товар_lower and "мыло" not in товар_lower) or
        ("для ногтей" in товар_lower and "набор" not in товар_lower and "," not in товар_lower) or
        ("краска для бровей" in товар_lower and "карандаш" not in товар_lower and "лайнер" not in товар_lower and "помад" not in товар_lower) or
        ("накладные" in товар_lower and "тушь" not in товар_lower) or
        (" для очищения" in товар_lower) or ("для объема волос" in товар_lower) or (" для обезжиривания" in товар_lower) or ("эталон микс" in товар_lower) or ("краска-бальзам для ресниц" in товар_lower) or
        ("для тату" in товар_lower) or ("для линии роста волос" in товар_lower) or
        ("энзим" in товар_lower and "не энзимная" not in товар_lower) or
        ("скраб" in товар_lower and "бров" not in товар_lower and "губ" not in товар_lower) or
        ("хна для" in товар_lower) or
        any(keyword in товар_lower for keyword in keywords)
    ):
        row['ГРУППА'] = "удалить"
        row['ПОДГРУППА'] = "удалить"
        row['КАТЕГОРИЯ'] = "удалить"
        row['ПОДКАТЕГОРИЯ'] = "удалить"

    return row

In [ ]:
df_updated_new = df_updated_with_learn.apply(update_row, axis=1)

In [ ]:
# Список категорий для удаления
categories_to_remove = [
    "Красота/Макияж/Глаза/Краска для бровей и ресниц",
    "Красота/Макияж/Брови/Краска для бровей и ресниц",
    "Красота/Макияж/Глаза/Накладные ресницы",
    "Красота/Макияж/Глаза/Зажимы для ресниц",
    "Красота/Макияж/Глаза/Клей для накладных ресниц",
    "Красота/Макияж/Глаза/Удалитель накладных ресниц",
    "Красота / Щетки косметические",
    "Красота / Средства для очистки спонжей и кистей",
    "Красота / Кремы",
    "Красота / Инструменты для бровей",
    "Красота / Нейтрализаторы испарений клея",
    "Красота / Аппликаторы",
    "Красота / Спонжи",
    "Красота / Трафареты для макияжа",
    "Красота / Инструменты для депиляции",
    "Красота / Кисти косметические",
    "Красота/Макияж/Брови/Инструменты для бровей",
    "Красота / Инструменты для бровей",
    "Красота / Пигменты",
    "Красота / Щетки косметические",
    "Красота/Макияж/Брови/Шампунь",
    "Красота/Макияж/Глаза/Праймеры для ресниц",
    "Красота/Макияж/Брови/Оксидант",
    "Красота/Макияж/Брови/Бритвы безопасные",
    "Красота/Макияж/Брови/Хна для волос",
    "Красота/Макияж/Лицо/Ремувер для удаления татуажа",
    "Красота/Макияж/Губы/Ремувер для удаления татуажа",
    "Красота/Макияж/Глаза/Ремувер для удаления татуажа",
    "Красота/Макияж/Брови/Ремувер для удаления татуажа",
    "Красота/Макияж/Брови/Точилки косметические",
    "Красота/Макияж/Глаза/Точилки косметические",
    "Красота/Макияж/Глаза/Аппликаторы",
    "Красота/Макияж/Лицо/Аппликаторы",
    "Красота/Макияж/Глаза/Трафареты для макияжа",
    "Красота/Макияж/Брови/Трафареты для макияжа",
    "Красота/Макияж/Лицо/Кисть косметическая",
    "Красота/Макияж/Глаза/Кисть косметическая",
    "Красота/Макияж/Брови/Кисть косметическая",
    "Красота/Макияж/Кисти для дизайна ногтей",
    "Красота/Макияж/Лицо/Косметические наборы для ухода",
    "Красота/Макияж/Брови/Смывка краски для волос",
    "Красота/Макияж/Глаза/Нейтрализаторы испарений клея",
    "Красота/Макияж/Брови/Пинцеты",
    "Красота / Точилки косметические",
    "Красота/Макияж/Глаза/Ресницы для наращивания",
    "Красота/Макияж/Глаза/Патчи",
    "Красота / Накладные ресницы",
    "Красота / Средства для очистки спонжей и кистей",
    "Красота / Щетки косметические",
    "Красота / Праймеры для ресниц",
    "Красота / Зажимы для ресниц",
    "Красота / Аппликаторы",
    "Красота / Сыворотки",
    "Красота / Удалитель накладных ресниц",
    "Красота / Закрепители для ресниц",
    "Красота / Трафареты для макияжа",
    "Красота / Клей для накладных ресниц",
    "Красота / Мочалки",
    "Красота / Смывка краски для волос",
    "Красота / Ремуверы для удаления татуажа",
    "Красота / Пенки",
    "Красота / Краски для бровей и ресниц",
    "Красота / Нейтрализаторы испарений клея",
    "Красота / Хна для волос",
    "Красота / Оксиданты",
    "Красота / Бальзамы",
    "Красота / Пилинг",
    "Красота / Кремы",
    "Красота / Временные татуировки и стразы",
    "Красота / Временные татуировки и стразы",
    "Красота / Бритвы безопасные",
    "Красота / Пинцеты",
    "Красота / Органайзеры для косметики",
    "Красота / Молочко",
    "Красота / Средства для снятия накладных ресниц",
    "Красота / Маски косметические",
    "Красота / Скрабы",
    "Красота / Косметические наборы для ухода",
    "Красота / Микробраши",
    "Красота / Зубные щетки",
    "Красота / Расчески",
    "Красота / Парфюмерные наборы",
    "Красота / Полоски для век",
    "Красота / Краски для тату",
    "Красота / Инструменты для депиляции",
    "Красота / Оттеночные бальзамы",
    "Красота / Бальзамы после бритья",
    "Красота / Бальзамы после бритья",
    "Красота / Маски тканевые косметические",
    "Красота / Инструменты для чистки ушей",
    "Красота / Салфетки косметические",
    "Красота / Лаки для ногтей",
    "Красота / Муссы",
    "Красота / Масла",
    "Красота / Маски оттеночные",
    "Красота / Праймеры для тела",
    "Красота / Кисти-сметки парикмахерские",
    "Красота / Кисти-сметки парикмахерские",
    "Красота / Воск для депиляции",
    "Красота / Спреи",
    "Красота / Патчи",
    "Красота / Эссенции",
    "Красота / Обезжириватели для ресниц",
    "Красота / Ватные диски",
    "Красота / Палетки косметические сменные",
    "Красота / Акриловые пудры",
    "Красота / Флюиды",
    "Красота/Макияж/Глаза/Закрепители для ресниц",
    "Красота/Макияж/Брови/Временные татуировки",
    "Красота/Макияж/Брови/Пилинг",
    "Красота/Макияж/Брови/Аксессуары для окрашивания волос",
    "Красота/Макияж/Лицо/Кисти-сметки парикмахерские",
    "Красота/Макияж/Расчески",
    "Красота/Макияж/Декор для маникюра",
    "Игрушки/Игрушечное оружие",
    "Красота/Макияж/Глаза/Наборы для наращивания ресниц",
    "Красота/Скрабы",
    "Красота/Маски косметические",
    "Красота/Маски тканевые косметические",
    "Красота/Ароматизаторы для бани",
    "Красота/Духи",
    "Красота / Мочалки",
    "Красота/Маникюрные наборы",
    "Красота/Тоники",
    "Красота/Пробники парфюмерные",
    "Красота/Макияж/Лицо/Средства для снятия макияжа",
    "Красота/Макияж/Глаза/Средства для снятия макияжа",
    "Красота/Кремы для бани",
    "Красота/Мочалки",
    "Красота/Органайзеры для косметики",
    "Красота/Краски для волос",
    "Красота/Кисти косметические",
    "Красота/Макияж/Лицо/Спонж"
    "Красота/Макияж/Лицо/Спонж"
    "Красота/Макияж/Глаза/Средство для очистки спонжей и кистей"
    "Красота/Макияж/Лицо/Средство для очистки спонжей и кистей"
]

In [ ]:
df_filtered = df_updated_new[~df_updated_with_learn['Категория'].isin(categories_to_remove)]

# Сохранение результатов
Результаты сохраняются в файл Excel, который затем можно скачать на локальный компьютер для дальнейшего анализа.

In [ ]:
# Сохраняем DataFrame в файл Excel
df_filtered.to_excel("Ноябрь.xlsx", index=False, engine='openpyxl')

In [ ]:

try:
    from google.colab import files
    files.download('Ноябрь.xlsx')
except ImportError:
    print("Функция загрузки файла не поддерживается в вашей среде")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>